In [7]:
import camelot
import pandas as pd

# Read all the tables across all pages

# Using Camelot
# List of Mining Companies
tables = camelot.read_pdf('SOCIETES_MINIERES.pdf', pages="all", flavor='lattice')
df = tables[0]

# Concatenating all the tables with pandas
df = pd.concat([table.df for table in tables], ignore_index=True)
df.head()

,0,1,2,3,4
0,Nom entreprise,Substance,Adresse Physique,Téléphone,Email
1,2M TRADING MULTISERVICES SARL,,"80, Cité Isra1, Hann Bel Air, Dakar, Sénégal",77 260 85 48,
2,2SBCI,calcaire,"Villa 477, unité 5,Parcelles Assainies, Keur M...",77 434 83 83,
3,2SH SALY SOFTWARE ET HARDWARE,,"Canal 4, en face Maison de la Douane, DAKAR, S...",77 439 28 61,
4,3S INTERNATIONAL,,"Jardin de Fann, Villa n°1, Fann Résidence, Dak...",,


In [8]:

df.columns = df.iloc[0]
df = df.drop(index=0).reset_index(drop=True)

df.drop_duplicates(inplace=True)
df['secteur'] = 'MINIER'
df['pays'] = 'SENEGAL'
df.head()

,Nom entreprise,Substance,Adresse Physique,Téléphone,Email,secteur,pays
0,2M TRADING MULTISERVICES SARL,,"80, Cité Isra1, Hann Bel Air, Dakar, Sénégal",77 260 85 48,,MINIER,SENEGAL
1,2SBCI,calcaire,"Villa 477, unité 5,Parcelles Assainies, Keur M...",77 434 83 83,,MINIER,SENEGAL
2,2SH SALY SOFTWARE ET HARDWARE,,"Canal 4, en face Maison de la Douane, DAKAR, S...",77 439 28 61,,MINIER,SENEGAL
3,3S INTERNATIONAL,,"Jardin de Fann, Villa n°1, Fann Résidence, Dak...",,,MINIER,SENEGAL
4,AAURIC HOLDINGS PTE.LTD,,"Km 18, Rte de RUFISQUE, BP 3835, MBAO, Sénégal",,,MINIER,SENEGAL


In [9]:
records = df.to_dict(orient='records')
for i in range(8):
    print(records[i])

{'Nom entreprise': '2M TRADING MULTISERVICES SARL', 'Substance': '', 'Adresse Physique': '80, Cité Isra1, Hann Bel Air, Dakar, Sénégal', 'Téléphone': '77 260 85 48', 'Email': '', 'secteur': 'MINIER', 'pays': 'SENEGAL'}
{'Nom entreprise': '2SBCI', 'Substance': 'calcaire', 'Adresse Physique': 'Villa 477, unité 5,Parcelles Assainies, Keur Massar, Dakar, \nSénégal', 'Téléphone': '77 434 83 83', 'Email': '', 'secteur': 'MINIER', 'pays': 'SENEGAL'}
{'Nom entreprise': '2SH SALY SOFTWARE ET HARDWARE', 'Substance': '', 'Adresse Physique': 'Canal 4, en face Maison de la Douane, DAKAR, Sénégal', 'Téléphone': '77 439 28 61', 'Email': '', 'secteur': 'MINIER', 'pays': 'SENEGAL'}
{'Nom entreprise': '3S INTERNATIONAL', 'Substance': '', 'Adresse Physique': 'Jardin de Fann, Villa n°1, Fann Résidence, Dakar, Sénégal', 'Téléphone': '', 'Email': '', 'secteur': 'MINIER', 'pays': 'SENEGAL'}
{'Nom entreprise': 'AAURIC HOLDINGS PTE.LTD', 'Substance': '', 'Adresse Physique': 'Km 18, Rte de RUFISQUE, BP 3835, MB

In [10]:
def split_phone_email(records, source_col="Email", phone_col="Téléphone", email_col="Email"):
    """
    Splits combined phone+email field into separate columns.
    Assumes phone comes first, email comes second.
    """
    for row in records:
        field = row.get(source_col, '').replace('\n','').strip()
        if field:
            parts = field.split()
            if len(parts) >= 2:
                phone = " ".join(parts[:-1]).strip()
                email = parts[-1].strip()

                row[phone_col] = phone
                row[email_col] = email

    return records

In [11]:
records = split_phone_email(records, 'Email', 'Téléphone', 'Email')


import re

email_pattern = re.compile(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}")

for row in records:
    field = row.get('Téléphone', '').strip()
    if field and email_pattern.search(field):
        # Append or set Email
        if row.get('Email'):
            row['Email'] = "/ ".join([row['Email'], field])
        else:
            row['Email'] = field

        # Remove the email from Téléphone
        row['Téléphone'] = email_pattern.sub("", row['Téléphone']).strip()


In [12]:
clean_df = pd.DataFrame(records)
clean_df.to_excel('SOCIETES_MINIERES.xlsx', index=False)